In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andradaolteanu/gtzan-dataset-music-genre-classification")

print("Path to dataset files:", path)

100%|██████████| 1.21G/1.21G [00:23<00:00, 56.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/versions/1


In [11]:
path = '/root/.cache/kagglehub/datasets/andradaolteanu/gtzan-dataset-music-genre-classification/versions/1/Data'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import librosa
import librosa.display
import IPython.display as ipd

print(os.listdir(path))

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print("Number of available GPUs:", device_count)
    for i in range(device_count):
        print("GPU", i, ":", torch.cuda.get_device_name(i))
else:
    print("GPU is not available")

test_data = pd.read_csv(path + '/features_3_sec.csv')
test_data.head()

['features_3_sec.csv', 'features_30_sec.csv', 'genres_original', 'images_original']
GPU is not available


(9990, 60)